In [1]:
import numpy as np
import pandas as pd
import sys
import os
import glob
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/walid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/walid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/walid/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
resnet_fc1000_train = pd.read_csv("data/features_train/features_resnet1000_train.csv", header = None, index_col = 0)
resnet_train = pd.read_csv("data/features_train/features_resnet1000intermediate_train.csv", header = None, index_col = 0)

resnet_fc1000_test = pd.read_csv("data/features_test/features_resnet1000_test.csv", header = None, index_col = 0)
resnet_test = pd.read_csv("data/features_test/features_resnet1000intermediate_test.csv", header = None, index_col = 0)

print(resnet_fc1000_train.shape, resnet_fc1000_test.shape)
print(resnet_train.shape, resnet_test.shape)

(10000, 1000) (2000, 1000)
(10000, 2048) (2000, 2048)


In [19]:
print(resnet_train[0:10])

                           1         2         3         4         5     \
0                                                                         
images_train/5373.jpg  0.460352  1.018530  0.283533  0.053866  1.071661   
images_train/984.jpg   1.498549  0.295754  0.266695  0.265754  0.793800   
images_train/7127.jpg  0.460160  0.570405  0.535643  0.073016  0.158414   
images_train/5293.jpg  0.626306  0.095554  0.175943  0.096139  0.063999   
images_train/3688.jpg  0.253012  1.743208  0.359000  0.486566  0.473813   
images_train/3340.jpg  0.920477  0.506810  0.334729  0.551220  0.201660   
images_train/4787.jpg  0.312581  2.718709  0.456360  0.133989  0.071897   
images_train/5707.jpg  0.992423  0.185353  0.040412  0.651217  0.514700   
images_train/1262.jpg  0.504020  1.136837  0.122905  0.255936  0.000000   
images_train/8355.jpg  0.153254  0.353655  0.475493  0.386336  0.167553   

                           6         7         8         9         10    \
0                       

In [5]:
def load_descriptions(path):
    data = []
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as description_file:
            label = os.path.splitext(os.path.basename(filename))[0]
            sentence = " ".join(line.strip() for line in description_file)
            new_row = (sentence, label) 
            data.append(new_row)

    dt = np.dtype([('sentence', object), ('label', object)])
    return(np.array(data, dtype = dt))

In [6]:
train_descriptions_path = "data/descriptions_train/"
test_descriptions_path = "data/descriptions_test/"

raw_train_data = load_descriptions(train_descriptions_path)
raw_test_data = load_descriptions(test_descriptions_path)

In [7]:
print(raw_train_data.shape, raw_test_data.shape)
print(raw_train_data[0:10])

(10000,) (2000,)
[('The skateboarder is putting on a show using the picnic table as his stage. A skateboarder pulling tricks on top of a picnic table. A man riding on a skateboard on top of a table. A skate boarder doing a trick on a picnic table. A person is riding a skateboard on a picnic table with a crowd watching.', '0')
 ('A bowl of soup that has some carrots, shrimp, and noodles in it. The healthy food is in the bowl and ready to eat. Soup has carrots and shrimp in it as it sits next to chopsticks. A tasty bowl of ramen is served for someone to enjoy. Bowl of Asian noodle soup, with shrimp and carrots.', '1')
 ('A bunch of luggage laying on an area rug. Several pieces of luggage on a floor with an area rug. The luggage is sitting on top of the persian rug. a bunch of travel bags sit on a carpet floor Several pieces of luggage that are laying on the floor.', '10')
 ('The browned cracked crust of a baked berry pie. A brown crust of pie with strawberry filing. The top of a pie look

In [8]:
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))
keep_pos_nouns = ['NN', 'NNS', 'NNP', 'NNPS']
keep_pos_all = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'VB', 'VBG', 'VBZ']
wnl = nltk.stem.WordNetLemmatizer()

def preprocess(data, pos_to_keep = None):
    
    out = data.copy()
    out['sentence'] = list(map(lambda x:x.lower(), out['sentence']))
    out['sentence'] = list(map(lambda x:''.join(ch for ch in x if ch not in punctuation), out['sentence']))
    out['sentence'] = list(map(lambda x:' '.join(w for w in x.split(' ') if w not in stop_words), out['sentence']))
    out['sentence'] = list(map(lambda x:' '.join(wnl.lemmatize(w) for w in x.split(' ')), out['sentence']))
    
    if pos_to_keep != None:
        out['sentence'] = list(map(lambda x:' '.join(w[0] for w in nltk.pos_tag(nltk.word_tokenize(x)) if w[1] in pos_to_keep), out['sentence']))
    
    return(out)

In [9]:
train_data = preprocess(raw_train_data, keep_pos_all)
test_data =  preprocess(raw_test_data, keep_pos_all)

In [10]:
print(train_data.shape, test_data.shape)
print(train_data[0:10])

(10000,) (2000,)
[('skateboarder putting show using picnic table stage skateboarder pulling trick top picnic table man riding skateboard top table skate boarder trick picnic table person riding skateboard picnic table crowd watching', '0')
 ('bowl soup carrot shrimp noodle healthy food bowl ready eat soup carrot shrimp sits chopstick tasty bowl ramen someone enjoy bowl asian noodle soup shrimp carrot', '1')
 ('bunch luggage laying area several piece luggage floor area rug luggage sitting top persian rug bunch travel bag sit carpet floor several piece luggage laying floor', '10')
 ('cracked crust berry brown crust pie strawberry filing top pie look good cooked fruit muffin image sort flaky pastery display', '100')
 ('sign clear view traffic light juice theory sign red traffic light bright sign hanging next traffic signal sign electrical post juice sign reading juice theory traffic light', '1000')
 ('cat sleeping duffel bag orange cat black duffel bag cat sitting bag small dog packes bag

In [11]:
word_dict = set()
for s in train_data['sentence']:
    words = s.split()
    for w in words:
        word_dict.add(w)

In [12]:
print(len(word_dict))
word_dict_list = list(word_dict)
word_dict_list.sort()
word_dict_list[0:20]

7038


['aa',
 'abandoned',
 'abdomen',
 'ability',
 'able',
 'abll',
 'abnormal',
 'aboard',
 'abook',
 'abou',
 'abraham',
 'abreast',
 'abstract',
 'abundance',
 'abundant',
 'abyss',
 'acacia',
 'academic',
 'accelerates',
 'accent']

In [13]:
def vectorize(train_data, test_data):
    
    Tfidf = TfidfVectorizer(vocabulary = word_dict, 
                            tokenizer = lambda str: str.split(" "))
    
    tr_d = [word for word in train_data['sentence'].tolist()] 
    te_d = [word for word in test_data['sentence'].tolist()] 
    
    X_train = Tfidf.fit_transform(tr_d)
    X_test = Tfidf.fit_transform(te_d)
    
    return(X_train, X_test)

In [14]:
X_train, X_test = vectorize(train_data, test_data)
print(X_train.shape, X_test.shape)

(10000, 7038) (2000, 7038)


In [15]:
print(train_data[0])
print(X_train[0])

('skateboarder putting show using picnic table stage skateboarder pulling trick top picnic table man riding skateboard top table skate boarder trick picnic table person riding skateboard picnic table crowd watching', '0')
  (0, 6783)	0.12053926958581132
  (0, 6619)	0.12127189129739563
  (0, 6414)	0.24667824208973552
  (0, 6319)	0.14277759161319625
  (0, 6091)	0.3853043493472325
  (0, 5802)	0.18503232966603594
  (0, 5522)	0.26959294300622977
  (0, 5521)	0.23031291251512154
  (0, 5519)	0.1277550086027136
  (0, 5433)	0.13522916887226213
  (0, 5027)	0.17069388821130024
  (0, 4796)	0.15739505742327461
  (0, 4772)	0.13727613164596256
  (0, 4449)	0.6510591890903679
  (0, 4412)	0.07473838331697687
  (0, 3575)	0.06230190293115674
  (0, 1490)	0.12361014704073833
  (0, 628)	0.158491376463526


In [16]:
print(word_dict_list[4449], word_dict_list[6091], word_dict_list[5522])

picnic table skateboarder


In [17]:
if not os.path.isdir("data/processed_descriptions"):
    os.mkdir("data/processed_descriptions")

np.save("data/processed_descriptions/train_descriptions.npy", X_train)
np.save("data/processed_descriptions/test_descriptions.npy", X_test)
